In [61]:
from scapy.sendrecv import sniff
import pandas as pd
import numpy as np
import time
from scapy.all import IP, TCP, UDP, send

# Function to read the CSV and send packets
def send_packets_from_csv(csv_file):
    # Read the CSV file
    df = pd.read_csv(csv_file)


    count = 0
    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():

        orig_ip = row['id.orig_addr']
        orig_port = row['id.orig_port']
        duration = row['duration_interval']
        pkt_count = row['orig_pkts_count']
        byte_count = row['orig_ip_bytes_count']

        # Add missing variables
        destination_ip = '172.20.10.2'
        # orig_ip = destination_ip
        protocol = np.random.choice(['UDP'])
        ttl = np.random.choice([64, 128, 255])
        flags = 'S' if protocol == 'TCP' else 'NONE'
        avg_payload_size = byte_count // pkt_count

        print(f"Sending {pkt_count} {protocol} packets from {orig_ip}:{orig_port} to {destination_ip} over {duration} seconds and length {byte_count}")

        # Calculate the interval between packets
        packet_interval = duration / pkt_count

        # Generate packets based on the packet count
        for i in range(pkt_count):
            # Construct the packet
            if protocol == 'TCP':
                packet = IP(src=orig_ip, dst=destination_ip) / TCP(sport=orig_port, dport=9999, flags=flags) / ("X" * avg_payload_size)
            elif protocol == 'UDP':
                packet = IP(src=orig_ip, dst=destination_ip) / UDP(sport=orig_port, dport=9999) / ("P" * avg_payload_size)

            # Set TTL value
            packet[IP].ttl = ttl

            # Send the packet
            send(packet)

            # Wait for the next packet based on the calculated interval
            if i < pkt_count - 1:
                time.sleep(packet_interval)

        if count == 3:
            break
        count += 1
    print('End')


flows_csv = "denormalized_flows.csv"
send_packets_from_csv(flows_csv)